In [2]:
# 1000 bits of floating point precision
RR = RealField(1000)

# log base 2 function
def log2(x): return log(x,2)

# helper function for computing proof size
def bits_to_bytes(x):
    byts = x/8
    KiB = 2^(10)
    MiB = 2^(20)
    GiB = 2^(30)
    TiB = 2^(40)
    if(byts < KiB):
        return "{} B".format(floor(byts))
    elif(byts < MiB):
        return "{} KiB".format(floor(byts/KiB))
    elif(byts < GiB):
        return "{} MiB".format(RR(byts/MiB).str(digits=3))
    elif(byts < TiB):
        return "{} GiB".format(RR(byts/GiB).str(digits=3))
    else:
        return "{} TiB".format(RR(byts/TiB).str(digits=3))


"""
    Compute proof sizes of FRI + Merkle.
    
    Parameters
        hash_out: output length of the hash function/random oracle
        ell: number of verifier repetitions during the query phase
        k: log2(message_length) / log2(degree_bound) (i.e., polynomials of degree < 2^k)
        n: log2(domain_size) (i.e., domain size = 2^n)
        F: lower bound or exact size of the finite field
"""
def proof_size(hash_out, ell, k, n, F):
    f_bits = (2*k*ell+1)*ceil(log2(F))
    hash_number = k + ell*( (n+1)*(n+2) - (n-k+1)*(n-k+2) )
    hash_bits = hash_out*hash_number
    
    total_bits = f_bits + hash_bits
    return(bits_to_bytes(total_bits))


"""
    Comptue batched proof sizes of FRI + Merkle. Assumes *good* batching
        (i.e., verifier sends distinct challenges for each polynomial 
        to batch).
    
    Parameters
        hash_out: output length of the hash function/random oracle
        ell: number of verifier repetitions during the query phase
        k: log2(message_length) / log2(degree_bound) (i.e., polynomials of degree < 2^k)
        n: log2(domain_size) (i.e., domain size = 2^n)
        F: lower bound or exact size of the finite field
        t: number of polynomials to batch
"""
def batch_proof_size(hash_out, ell, k, n, F, t):
    f_bits = (2*ell*(k+t)+1)*ceil(log2(F))
    hash_number = k + ell*( (n+1)*(n+2) - (n-k+1)*(n-k+2) ) + t*(1 + 2*ell*(n+1))
    hash_bits = hash_out * hash_number
    
    total_bits = f_bits + hash_bits
    return(bits_to_bytes(total_bits))




#### PART 1 CALCULATIONS ####

## The following 3 functions are for calculating the tradeoffs
##  between adversarial queries Q and achievable soundness
##  error.

## Calculation under provable RBR Soundness of FRI
##   eps = max{ ((m+1/2)^7 * (2^{k}/rho)^2)/ (3(rho)^(3/2)*F)  , (1-delta)^ell / 2^z }
##   delta = 1-sqrt(rho)(1+1/2m)
## Parameters:
##   F = Field size
##   rho = RS code rate
##   k = log(degree bound)
##   m = parameter for johnson proximity gap, at least 3
##   lgQ_list = list containing log of adversarial query bounds
##   kappa = RO hash output length (in bits)
##   ell = number of verifier queries during FRI Query phase
##   z = # grinding bits
def provable_calcs(F, rho, k, m, lgQ_list, kappa, ell, z=0): 
    if(m < 3):
        print("ERROR: proximity gap m < 3. Aborting...")
        return
    delta = 1-sqrt(rho)*(1+1/(2*m))
    N = (2^k)/rho
    rbr = max( ((m+1/2)^7 * N^2) / ( 3*(rho^(3/2))*F ), (((1-delta)^ell)/2^z)  )
    for i in lgQ_list:
        Q = 2^i
        nu1 = log2( 1 / (2*Q*rbr) ) 
        nu2 = log2( 2^(kappa-1) / (3*(Q^2+1)) )
        nu = floor(min(nu1,nu2))
        proof_len = proof_size(kappa, ell, k, log2(N), F)
        print("$Q=2^{",i,"}$,","({}, {}, {}, {})".format(nu, RR(delta).str(digits=5), ell, proof_len))
    return


## Calculation under conjectured RBR Soundness of FRI
##   eps = max{ 1/F  , (1-delta)^ell / 2^z }
##   delta = 1-rho
## Parameters:
##   F = Field size
##   rho = RS code rate
##   k = log(degree bound)
##   lgQ_list = list containing log of adversarial query bounds
##   kappa = RO hash output length (in bits)
##   ell = # verifier queries during FRI Query phase
##   z = # grinding bits
def conj_calcs_2(F, rho, k, lgQ_list, kappa, ell, z=0):
    delta = 1-rho
    N = (2^k)/rho
    rbr = max(1/F, ((1-delta)^ell / 2^z))
    for i in lgQ_list:
        Q = 2^i
        nu1 = log2( 1 / (2*Q*rbr) ) 
        nu2 = log2( 2^(kappa-1) / (3*(Q^2+1)) )
        nu = floor(min(nu1,nu2))
        proof_len = proof_size(kappa, ell, k, log2(N), F)
        print("$Q=2^{",i,"}$,","({}, {}, {}, {})".format(nu, RR(delta).str(digits=5), ell, proof_len))
    return


## In the next 2 functions, I calculate the bits of security
##   that are achievable certain sets of parameters.
def bits_of_security_provable(F, rho, k, m, kappa, ell, z=0):
    delta = 1-sqrt(rho)*(1+1/(2*m))
    N = (2^k)/rho
    rbr = max( ((m+1/2)^7 * N^2) / ( 3*(rho^(3/2))*F ), (((1-delta)^ell)/2^z)  )
    BoS1 = -log2(rbr) - 1
    lam = floor(BoS1)
    lgQ = floor(log2( (2^(kappa - lam - 1) / 3) - 1 ))
    print("Bits of security (Provable): $\\lambda \\leq {}$, $Q \\geq 2^{}$".format(lam,lgQ))
    return


def bits_of_security_conj2(F, rho, k, kappa, ell, z=0):
    delta = 1-rho
    rbr = max( 1/F, ((1-delta)^ell) / 2^z)
    BoS1 = -log2(rbr) - 1
    lam = floor(BoS1)
    lgQ = floor(log2( (2^(kappa - lam - 1) / 3) - 1 ))
    print("Bits of security (Conjecture 2): $\\lambda \\leq {}$, $Q \\geq 2^{}$".format(lam, lgQ))
    return

## The following function calculates ell such that
## the query phase achieves lambda bits of security
## Parameters:
##   delta: proximity parameter
##   z: grinding bits
##   lam: target bits of security
def ell_calc(delta, z, lam):
    return ceil( (lam + 1 - z) / log2(1/(1-delta)) )



### Function to run above calculations of a system ###
def run_calcs(name, lgQ_list, kappa, F, rho, k, ell_list, z=0, m=3):
    print()
    print("=== {} ===".format(name))
    
    print("Provable Security Calculations")
    
    ell = ell_list[0]
    provable_calcs(F, rho, k, m, lgQ_list, kappa, ell, z)
    bits_of_security_provable(F, rho, k, m, kappa, ell, z)
    
    if(z>0): # calcs without grinding
        print("-- without grinding --")
        provable_calcs(F, rho, k, m, lgQ_list, kappa, ell_list[0])
        bits_of_security_provable(F, rho, k, m, kappa, ell)
        
    print()
    print("Conjectured Security Calculations")
    
    if(len(ell_list)> 1):
        ell = ell_list[1]
    conj_calcs_2(F, rho, k, lgQ_list, kappa, ell, z)
    bits_of_security_conj2(F, rho, k, kappa, ell, z)
    
    if(z>0): # calcs without grinding
        print("-- without grinding -- ")
        conj_calcs_2(F, rho, k, lgQ_list, kappa, ell)
        bits_of_security_conj2(F, rho, k, kappa, ell)
    
    print()
    print("+=+=+=+=+=+=+=+=+=+=+=+=+")


    

### set targets for number of RO queries ###
lgQ_list = [20*i for i in range(1,7)]

# hash output of RO
kappa = 256

    
### calculations for RISC Zero Parameters ###

# RISC Zero: 1
name = "RISC Zero Parameters: 1"
F = (2^64 -2^32+1)^2 # goldilocks field deg 2 ext
rho = 1/4
k = 24
ell = [50]

run_calcs(name, lgQ_list, kappa, F, rho, k, ell)


# RISC Zero: 2
name = "RISC Zero Parameters: 2"
F = (2^31 - 2^27 +1)^4 # baby bear deg 4 ext
rho = 1/4
k = 24
ell = [50]

run_calcs(name, lgQ_list, kappa, F, rho, k, ell)

### calculations for Plonky2 Parameters, Part 1 ###
print()
name ="Plonky2 Parameters 1"
F = (2^64 - 2^32 + 1)^2 # goldilocks field deg 2 ext
rho = 1/2
k = 31
ell = [84]
z=16

run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

    

### calculations for Plonky2 Parameters, Part 2 ###
name = "Plonky2 Parameters 2"
F = (2^64 - 2^32 + 1)^2
rho = 1/8
k = 29
ell = [28]
z=16

run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)


### calculations for lambdaworks parameters ###
F = 2^(251)
rho = 1/4
k = log2((2^(40)) * rho)
z=20
ell = [80, 31]

name = "lambdaworks Parameters: 80-bits" 

run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

name="lambdaworks Paramegers: 100-bits"
ell=[104, 41]
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)




name = "lambdaworks Parameters: 128-bits"
ell=[140, 55]
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)



### calculations for Stone Prover Parameters ###

name = "Stone Prover Parameters: k=13"
F = 2^251 
rho = 1/16
k = 13
ell = [18]
z=24
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

name = "Stone Prover Parameters: k=30"
k = 30
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)


### dYdX calculations ###
name = "dYdX Parameters"
F = 2^(251)
k = 31
z = 32
ell = [12]
rho=1/16
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

### miden-vm ###
name = "miden-vm: 96-bits"
F = (2^(64)-2^(32)+1)^2
ell = [27]
z=16
rho=1/8
k = 30
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

name="miden-vm: 128-bits"
F= (2^(64)-2^(32)+1)^3
z=21
rho=1/16
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

### boojam default/benchmark parameters ###
name = "boojam: default parameters"
F = (2^(64)-2^(32)+1)^2
z=20
rho=1/4
ell=[40]
k = 19
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)

name = "boojam: benchmark parameters"
rho = 1/8
ell=[34]
run_calcs(name, lgQ_list, kappa, F, rho, k, ell)

### SHARP Verifier parameters ###
name = "SHARP Verifier"
F = 2^(251)
rho = 1/16
ell = [16]
k = 27
z = 32
run_calcs(name, lgQ_list, kappa, F, rho, k, ell, z)



=== RISC Zero Parameters: 1 ===
Provable Security Calculations
$Q=2^{ 20 }$, (17, 0.41667, 50, 1.17 MiB)
$Q=2^{ 40 }$, (-3, 0.41667, 50, 1.17 MiB)
$Q=2^{ 60 }$, (-23, 0.41667, 50, 1.17 MiB)
$Q=2^{ 80 }$, (-43, 0.41667, 50, 1.17 MiB)
$Q=2^{ 100 }$, (-63, 0.41667, 50, 1.17 MiB)
$Q=2^{ 120 }$, (-83, 0.41667, 50, 1.17 MiB)
Bits of security (Provable): $\lambda \leq 37$, $Q \geq 2^216$

Conjectured Security Calculations
$Q=2^{ 20 }$, (79, 0.75000, 50, 1.17 MiB)
$Q=2^{ 40 }$, (59, 0.75000, 50, 1.17 MiB)
$Q=2^{ 60 }$, (39, 0.75000, 50, 1.17 MiB)
$Q=2^{ 80 }$, (19, 0.75000, 50, 1.17 MiB)
$Q=2^{ 100 }$, (-1, 0.75000, 50, 1.17 MiB)
$Q=2^{ 120 }$, (-21, 0.75000, 50, 1.17 MiB)
Bits of security (Conjecture 2): $\lambda \leq 99$, $Q \geq 2^154$

+=+=+=+=+=+=+=+=+=+=+=+=+

=== RISC Zero Parameters: 2 ===
Provable Security Calculations
$Q=2^{ 20 }$, (17, 0.41667, 50, 1.17 MiB)
$Q=2^{ 40 }$, (-3, 0.41667, 50, 1.17 MiB)
$Q=2^{ 60 }$, (-23, 0.41667, 50, 1.17 MiB)
$Q=2^{ 80 }$, (-43, 0.41667, 50, 1.17 Mi

$Q=2^{ 20 }$, (25, 0.58752, 27, 1013 KiB)
$Q=2^{ 40 }$, (5, 0.58752, 27, 1013 KiB)
$Q=2^{ 60 }$, (-15, 0.58752, 27, 1013 KiB)
$Q=2^{ 80 }$, (-35, 0.58752, 27, 1013 KiB)
$Q=2^{ 100 }$, (-55, 0.58752, 27, 1013 KiB)
$Q=2^{ 120 }$, (-75, 0.58752, 27, 1013 KiB)
Bits of security (Provable): $\lambda \leq 45$, $Q \geq 2^208$
-- without grinding --
$Q=2^{ 20 }$, (13, 0.58752, 27, 1013 KiB)
$Q=2^{ 40 }$, (-7, 0.58752, 27, 1013 KiB)
$Q=2^{ 60 }$, (-27, 0.58752, 27, 1013 KiB)
$Q=2^{ 80 }$, (-47, 0.58752, 27, 1013 KiB)
$Q=2^{ 100 }$, (-67, 0.58752, 27, 1013 KiB)
$Q=2^{ 120 }$, (-87, 0.58752, 27, 1013 KiB)
Bits of security (Provable): $\lambda \leq 33$, $Q \geq 2^220$

Conjectured Security Calculations
$Q=2^{ 20 }$, (76, 0.87500, 27, 1013 KiB)
$Q=2^{ 40 }$, (56, 0.87500, 27, 1013 KiB)
$Q=2^{ 60 }$, (36, 0.87500, 27, 1013 KiB)
$Q=2^{ 80 }$, (16, 0.87500, 27, 1013 KiB)
$Q=2^{ 100 }$, (-4, 0.87500, 27, 1013 KiB)
$Q=2^{ 120 }$, (-24, 0.87500, 27, 1013 KiB)
Bits of security (Conjecture 2): $\lambda \leq

In [ ]:
RR = RealField(1000)


def log2(x): return log(x,2)

def bits_to_bytes(x):
    byts = x/8
    KiB = 2^(10)
    MiB = 2^(20)
    GiB = 2^(30)
    TiB = 2^(40)
    if(byts < KiB):
        return "{} B".format(floor(byts))
    elif(byts < MiB):
        return "{} KiB".format(floor(byts/KiB))
    elif(byts < GiB):
        return "{} MiB".format(RR(byts/MiB).str(digits=3))
    elif(byts < TiB):
        return "{} GiB".format(RR(byts/GiB).str(digits=3))
    else:
        return "{} TiB".format(RR(byts/TiB).str(digits=3))

        

def proof_size(hash_out, ell, k, n, F):
    f_bits = (2*k*ell+1)*ceil(log2(F))
    hash_number = k + ell*( (n+1)*(n+2) - (n-k+1)*(n-k+2) )
    hash_bits = hash_out*hash_number
    
    total_bits = f_bits + hash_bits
    return(bits_to_bytes(total_bits))


def batch_proof_size(hash_out, ell, k, n, F, t):
    f_bits = (2*ell*(k+t)+1)*ceil(log2(F))
    hash_number = k + ell*( (n+1)*(n+2) - (n-k+1)*(n-k+2) ) + t*(1 + 2*ell*(n+1))
    hash_bits = hash_out * hash_number
    
    total_bits = f_bits + hash_bits
    return(bits_to_bytes(total_bits))




def calc_bound_provable(nu, lgQ_list, F_list, k_list, rho_list, kappa):
    for lgQ in lgQ_list:
        Q = 2^(lgQ) # number of RO queries we're allowing an adversary to make
        hash_out = kappa # hash output length
        print("nu =", nu)
        print("\\hhline{|t:==:t|}")
        out_head = "\\multicolumn{2}{||c||}{$\\nu = " + str(nu) + "$} "
        out_head = out_head + "& \\multicolumn{4}{c}{\\cref{cor:fs-fri}} \\\\\\hhline{||~~||----}\n"
        out_head = out_head + "\\multicolumn{2}{||c||}{$Q = 2^{" +str(lgQ) + "}$} "
        for rho in rho_list:
            out_head = out_head + "& $\\rho = " + str(rho) + "$ "
        out_head = out_head + "\\\\\\hhline{|b:==:b|*{4}{=|}}"
        print(out_head)
        for F in F_list:
            #print("\tField of size 2^{f}".format(f=log(F,2)))
            out = "{} ".format(log2(F))
            out2 = ""
            for k in k_list:
                #print("\tDomain of size 2^{n}".format(n=log(N,2)))
                out2 = out2 + "& {} ".format(k)
                for rho in rho_list:
                    delta = 0 # FRI proximity parameter to calculate
                    valid = True # check if computation below has valid constraints
                    m=0 # Johnson proximity parameter to compute
                    n=floor(log2((2^k)/rho)) # log2 size of the evaluation domain/codeword length
                    N = 2^n
                    
                    # do calculations for provable levels of security
                    m = floor( ((3*(rho^(3/2))*F) / (N^2 * Q * (2^(nu+1))))^(1/7) - (1/2) )
                    if(3 > m):
                        valid = False # constraint is violated
                    else:
                        delta = 1-sqrt(rho)*(1+(1/(2*m)))


                    if(valid):
                        #ell = ceil( log2( (1/(Q*2^(nu+1))) ) / log2(1-delta) )
                        #rbr = (((m+1/2)^7) * ((2^n)^2)) / ( 3*((rho)^(3/2))*F )
                        upper = 1/(Q*(2^(nu+1)))
                        ell = ceil( log2(upper) / log2(1-delta) )
                        #print(ell)
                        #print(ceil(log2(upper)/log2(1-delta)))
                        out2 = out2 + "& \\small({}, {}, {}) ".format(RR(delta).str(digits=3),ell,proof_size(hash_out, ell, k, n, F))
                    else:
                        out2 = out2 + "& \\cellcolor{lightgray} "

                out2 = out2 + "\\\\\n"
            print(out + out2[:-1] + "\\hhline{*{6}{-}}")
        out_last = "$\\log|\\bbF|$ & $k$ & \\multicolumn{4}{c}{$(\\delta, \\ell, |\\pi|)$} \\\\"
        print(out_last)
        print()
    return



## calculation for ethSTARK conjectured parameters
def calc_bound_conj_2(nu, lgQ_list, F_list, k_list, rho_list, kappa):
    for lgQ in lgQ_list:
        hash_out = kappa # hash output length
        print("nu =", nu)
        print("\\hhline{|t:==:t|}")
        out_head = "\\multicolumn{2}{||c||}{$\\nu = " + str(nu) + "$} "
        out_head = out_head + "& \\multicolumn{4}{c}{\\cref{conj:fri-2}} \\\\\\hhline{||~~||----}\n"
        out_head = out_head + "\\multicolumn{2}{||c||}{$Q = 2^{" +str(lgQ) + "}$} "
        for rho in rho_list:
            out_head = out_head + "& $\\rho = " + str(rho) + "$ "
        out_head = out_head + "\\\\\\hhline{|b:==:b|*{4}{=|}}"
        print(out_head)
        for F in F_list:
            out = "{} ".format(log2(F))
            out2 = ""
            Q = 0
            lgQ_valid = floor(log2(F/2^(nu+1)))
            valid = True
            if(lgQ > lgQ_valid):
                valid = False
            else:
                Q = 2^(lgQ)

            for k in k_list:
                out2 = out2 + "& {} ".format(k)
                for rho in rho_list:
                    delta = 1-rho # FRI proximity parameter 
                    n=floor(log2((2^k)/rho)) # log2 size of the evaluation domain/codeword length
                    N = 2^n

                    if(valid):
                        ell = ceil( log2( (1/(Q*2^(nu+1))) ) / log2(1-delta) )
                        out2 = out2 + "& \\small({}, {}, {}) ".format(RR(delta).str(digits=3),ell,proof_size(hash_out, ell, k, n, F))
                    else:
                        out2 = out2 + "& \\cellcolor{lightgray} "

                out2 = out2 + "\\\\\n"
            print(out + out2[:-1] + "\\hhline{*{6}{-}}")
        out_last = "$\\log|\\bbF|$ & $k$ & \\multicolumn{4}{c}{$(\\delta, \\ell, |\\pi|)$} \\\\"
        print(out_last)
        print()
    return


#### PART 1 CALCULATIONS ####
# target field sizes
F_list = [2^i for i in [128, 192, 256]]

# log of message lengths
k_list = [i for i in [10, 15, 20, 25]] 

# target rates
rho_list = [1/2^i for i in [1, 2, 3, 4]] 

# target security levels
nu_list = [20,40,60,80] 

# hash output length
kappa = 256 


lgQ_list = [60, 80]
## compute bounds for provable security and print tables
print("===== Provable Security Calculations =====")
for nu in nu_list:
    Q_upper = floor( sqrt( ((2^(kappa-nu-1))/3) - 1 )) # upper bound on number of RO queries
    lgQ_upper = floor(log2(Q_upper))
    print("$2^{",str(lgQ_upper),"} \\leq Q < 2^{", str(lgQ_upper+1), "}$") # print powers of 2 range of Q_upper
    #lgQ_list = [floor(lgQ_upper/10)*10-i for i in [30,20,10,0]]
    
    # calculate bounds and print tables
    calc_bound_provable(nu, lgQ_list, F_list, k_list, rho_list, kappa)

## compute bounds for conjectured security from [BCI+20a] and print tables
print("===== Conjectured Security Calculations 1 =====")
for nu in nu_list:
    Q_upper = floor( sqrt( ((2^(kappa-nu-1))/3) - 1 )) # upper bound on number of RO queries
    lgQ_upper = floor(log2(Q_upper))
    print("$2^{",str(lgQ_upper),"} \\leq Q < 2^{", str(lgQ_upper+1), "}$") # print powers of 2 range of Q_upper
    #lgQ_list = [floor(lgQ_upper/10)*10-i for i in [30,20,10,0]]
    
    # calculate bounds and print tables
    calc_bound_conj_1(nu, lgQ_list, F_list, k_list, rho_list, kappa)

## compute bounds for conjectured security from ethSTARK and print tables
print("===== Conjectured Security Calculations 2 =====")
for nu in nu_list:
    Q_upper = floor( sqrt( ((2^(kappa-nu-1))/3) - 1 )) # upper bound on number of RO queries
    lgQ_upper = floor(log2(Q_upper))
    print("$2^{",str(lgQ_upper),"} \\leq Q < 2^{", str(lgQ_upper+1), "}$") # print powers of 2 range of Q_upper
    
    calc_bound_conj_2(nu, lgQ_list, F_list, k_list, rho_list, kappa)
